# Datasets 2018
Para cada año tenemos 3 tablas a inspeccionar, limpiar y unir:
- `title_basics`: pública y descargada de IMDb
- `title_ratings`: pública y descargada de IMDb
- `movies`: propia, escrapeada de la web de IMDb para todas las películas del año correspondiente

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
anno = 2018

In [3]:
# Tabla descargada de IMDb

title_basics = pd.read_csv('../data/imdb/title_basics.tsv', sep='\t', na_values='\\N')
print(title_basics.shape)
title_basics.head()

C:\Users\casiopa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(8084314, 9)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1.0,"Comedy,Short"


In [4]:
# Tabla descargada de IMDb

title_ratings = pd.read_csv('../data/imdb/title_ratings.tsv', sep='\t')
print(title_ratings.shape)
title_ratings.head()

(1171920, 3)


,tconst,averageRating,numVotes
0,tt0000001,5.7,1722
1,tt0000002,6.0,211
2,tt0000003,6.5,1484
3,tt0000004,6.1,124
4,tt0000005,6.2,2286


In [5]:
# Películas de escrapeadas de la web IMDb

movies = pd.read_csv('../data/web_imdb/movies_df_'+ str(anno) + '.csv', sep=';', na_values=["[]", "['']", "['', '']", "['', '', '']"])
print(movies.shape)
movies.head()

(17819, 20)


,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,countries,companies,imdb_rating,metascore,popularity,awards,budget,gross_us_canada,opening_us_canada,gross_world
0,tt0069049,Al otro lado del viento,The Other Side of the Wind,2018.0,13,2h 2min,['Orson Welles'],"['Orson Welles', 'Oja Kodar']","['John Huston', 'Oja Kodar', 'Peter Bogdanovich']",['Drama'],"['France', 'Iran', 'United States']","['Royal Road Entertainment', ""Les Films de l'A...",6.8,78.0,NaN,Awards\n9 wins & 9 nominations,NaN,NaN,NaN,NaN
1,tt0111414,A Thin Life,NaN,2018.0,TV-PG,1h 15min,['Frank Howson'],['Frank Howson'],['Tommy Dysart'],['Comedy'],['Australia'],['Boulevard Films'],NaN,NaN,NaN,NaN,"A$100,000",NaN,NaN,NaN
2,tt0137818,Housesitter: The Night They Saved Siegfried's ...,NaN,2018.0,NaN,NaN,['Robin Nuyen'],"['Richard Gasparian', 'Robin Nuyen']","['Stephen James Carver', 'Richard Gasparian', ...",NaN,['United States'],['Basement Productions'],4.4,NaN,NaN,Awards\n1 win & 1 nomination,NaN,NaN,NaN,NaN
3,tt0170651,T.G.M. - osvoboditel,NaN,2018.0,NaN,NaN,['Vera Chytilová'],NaN,NaN,['Documentary'],['Czech Republic'],NaN,6.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0192528,Reverse Heaven,NaN,2018.0,TV-MA,1h 44min,['Stuart Paul'],['Stuart Paul'],"['Cheryl M. Lynn', 'David Carter', 'Bill Colla...",['Drama'],['United States'],['SP Sales Worldwide'],4.0,NaN,NaN,NaN,"$5,000,000",NaN,NaN,NaN


In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17819 entries, 0 to 17818
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   imdb_id            17819 non-null  object 
 1   title              17794 non-null  object 
 2   original_title     794 non-null    object 
 3   year               17793 non-null  float64
 4   certificate        2675 non-null   object 
 5   duration           2627 non-null   object 
 6   directors          17244 non-null  object 
 7   writers            15712 non-null  object 
 8   stars              11746 non-null  object 
 9   genres             17077 non-null  object 
 10  countries          17522 non-null  object 
 11  companies          12864 non-null  object 
 12  imdb_rating        9669 non-null   float64
 13  metascore          859 non-null    float64
 14  popularity         245 non-null    object 
 15  awards             4688 non-null   object 
 16  budget             489

---
## Número de películas

En primer lugar verificar si hay una discrepancia entre el número de películas obtenidas de title_basics.tsv de IMDb
y el número de películas resultantes del scrapping.

In [7]:
# Cargar las películas de la base de datos de IMDb en una lista

imdb_ids = title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)]
imdb_ids = list(imdb_ids['tconst'])
len(imdb_ids)

17819

In [8]:
scrap_ids = list(movies['imdb_id'])
len(scrap_ids)

17819

In [9]:
len(movies.drop_duplicates())

17819

---
## 1. Bd de IMDb: `title_basics`

### Variable `runtimeMinutes`  

In [10]:
title_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8084314 entries, 0 to 8084313
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   tconst          object 
 1   titleType       object 
 2   primaryTitle    object 
 3   originalTitle   object 
 4   isAdult         float64
 5   startYear       float64
 6   endYear         float64
 7   runtimeMinutes  object 
 8   genres          object 
dtypes: float64(3), object(6)
memory usage: 555.1+ MB


#### Convertir `runtimeMinutes` en float
La siguiente función convierte los valores en float si se puede de manera natural, y si no se puede los imprime y convierte a nulo.

In [11]:
def to_float(n):
    try:
        n = float(n)
        return n
    except:
        print(n)
        return np.nan

In [12]:
title_basics.runtimeMinutes = title_basics.runtimeMinutes.apply(to_float)
title_basics.runtimeMinutes

Reality-TV
Documentary
Talk-Show
Game-Show
Reality-TV
Animation,Comedy,Family
Reality-TV


0           1.0
1           5.0
2           4.0
3          12.0
4           1.0
           ... 
8084309     NaN
8084310     NaN
8084311     NaN
8084312    27.0
8084313    10.0
Name: runtimeMinutes, Length: 8084314, dtype: float64

#### Eliminar de `title_basics` los registros con outliers en `runtimeMinutes`

In [13]:
# Buscar outliers mayores de 1.000 minutos y menores 15
# El resltado es un listado de películas fake, cortos o compendios de pelis.
# Eliminar todos estos registros del scrapping

title_basics = title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)]

title_basics = title_basics[~((title_basics.runtimeMinutes<15) | (title_basics.runtimeMinutes>500))]
title_basics.shape

(17811, 9)

In [14]:
# Una vez eliminados los outliers estos son los estadísticos:

title_basics.runtimeMinutes.describe().round(2)

count    13887.00
mean        90.81
std         26.62
min         17.00
25%         75.00
50%         90.00
75%        103.00
max        495.00
Name: runtimeMinutes, dtype: float64

---
## 2. Bd de IMDb: `title_ratings`

In [15]:
title_ratings = pd.read_csv('../data/imdb/title_ratings.tsv', sep='\t')
print(title_ratings.shape)
title_ratings.head()

(1171920, 3)


,tconst,averageRating,numVotes
0,tt0000001,5.7,1722
1,tt0000002,6.0,211
2,tt0000003,6.5,1484
3,tt0000004,6.1,124
4,tt0000005,6.2,2286



---
## 3. Bd propia escrapeada del portal IMDb: `movies`

## `year`
La base de datos de IMDb descargada tiene algunas discrepancias en cuanto al año de las películas, no coindice el año en la base de datos descargada de IMDb con el año del portal web, en el caso del año 2019 ocurre en 12 registros. Existe algún caso en el que tampoco coincide el año de la película en el portal Metacritic. Hay varias formas de afrontar estas discrepancias, en este caso, y por simplificación, utilizaré el año de la base de datos de IMDb.
Hay también algunos registros en la base de datos con valores nulos para el año y demás columnas, y serán eliminados (47 registors en total).

In [16]:
# Películas escrapeadas con año diferente
len(movies[(movies['year']!=anno) & movies['year'].notnull()])

22

In [17]:
movies[(movies['year']!=anno) & movies['year'].notnull()]

,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,countries,companies,imdb_rating,metascore,popularity,awards,budget,gross_us_canada,opening_us_canada,gross_world
10,tt0800325,The Dirt,NaN,2019.0,18,1h 47min,['Jeff Tremaine'],"['Rich Wilkes', 'Amanda Adelson', 'Tommy Lee']","['Douglas Booth', 'Iwan Rheon', 'Machine Gun K...","['Biography', 'Comedy', 'Drama']",['United States'],"['10th Street Entertainment', 'LBI Entertainme...",7.0,39.0,"1,095",Awards\n1 win,NaN,NaN,NaN,NaN
11,tt0825334,Caravaggio and My Mother the Pope,NaN,2017.0,NaN,NaN,['Gladys Florence'],['Gladys Florence'],"['Gladys Florence', 'Gladys Rose', 'Tom Troupe']","['Comedy', 'Drama']",['United States'],['Giadeyes Productions'],8.4,NaN,NaN,NaN,"$3,000,000",NaN,NaN,NaN
2255,tt13098938,Kattumaram,NaN,2019.0,NaN,NaN,['Swarnavel Eswaran'],['Swarnavel Eswaran'],"['Mysskin', 'Preeti Karan', 'Anusha Prabhu']",['Drama'],['India'],NaN,6.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3036,tt2354065,Prahala,NaN,2022.0,NaN,NaN,['Beni Adam'],['Beni Adam'],"['Beni Adam', 'Vica Kerekes', 'Tatiana Pauhofo...",['Drama'],['Canada'],"['Beni Adam Pictures', 'Beni Adam']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3336,tt3595744,The Cardinal Rule,NaN,2016.0,NaN,NaN,['Chaun Pinkston'],['Chaun Pinkston'],"['Renee Chahoy', 'Elizabeth Cooper', 'Ashley C...","['Action', 'Crime', 'Drama']",['United States'],['Seedtime Productions'],6.6,NaN,NaN,NaN,"$120,000",NaN,NaN,NaN
3460,tt3896004,On Golden Years,NaN,2014.0,NaN,NaN,['Tirlok Malik'],"['Iggy Ignatius', 'Tirlok Malik']","['Ranjit Chowdhry', 'Jyoti Singh', 'Reeves Leh...","['Comedy', 'Drama', 'Romance']",['United States'],NaN,NaN,NaN,NaN,Awards\n1 win,NaN,NaN,NaN,NaN
4111,tt5068978,Romancing Brazil,NaN,2021.0,NaN,NaN,['Lucas Amberg'],"['Lucas Amberg', 'Andrew Deutsch']","['Talia Shire', 'Elliott Gould', 'Mariana Xime...","['Drama', 'Family', 'Romance']",['Brazil'],['Amberg Filmes'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5198,tt6102066,The Confession Project,NaN,2020.0,NaN,NaN,['Humberto Castro'],['Humberto Castro'],"['Jon Erik Castro', 'Lorraine Sanchez', 'Humbe...","['Short', 'Crime', 'Thriller']",['United States'],['Award Pictures'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7004,tt7112784,"Love, Gloria",NaN,2012.0,NaN,NaN,['Nick Scown'],['Nick Scown'],"['Heather McComb', 'Danny Woodburn', 'Stephani...","['Comedy', 'Crime', 'Drama']",['United States'],['RMS Films'],7.2,NaN,NaN,NaN,"$300,000",NaN,NaN,NaN
7089,tt7141096,Día de lluvia en Nueva York,A Rainy Day in New York,2019.0,7,1h 32min,['Woody Allen'],['Woody Allen'],"['Timothée Chalamet', 'Elle Fanning', 'Liev Sc...","['Comedy', 'Romance']",['United States'],"['Gravier Productions', 'Perdido Productions']",6.5,38.0,"1,311",Awards\n3 wins,NaN,NaN,NaN,"$22,002,504"


In [18]:
title_basics[title_basics['tconst']=='tt10461238']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


#### Eliminar del df escrapeado las películas de años posteriores a 2019
(son pelis futuras que se han colado)

In [19]:
movies = movies[~(movies['year']>2019)]
len(movies)

17811

#### Eliminar del df escrapeado las películas sin año
(son pelis sin información o url en el portal)

In [20]:
# Películas escrapeadas con año null, no tienen casi información, serán eliminadas
movies = movies[movies['year'].notnull()]
len(movies)

17785

## `imdb_rating`
Esta variable será sustituida por la de la base de datos `title_rating` descargada de IMDb

In [21]:
movies[['imdb_rating']].describe().round(2)

,imdb_rating
count,9668.00
mean,6.15
std,1.48
min,1.00
25%,5.30
50%,6.30
75%,7.20
max,10.00


## `metascore`
Comparing to: https://www.metacritic.com/feature/best-movies-released-in-2019

Hay pequeñas discrepancias en el listado de películas, debido a las fechas de estreno. Metacritic considera la fecha de estreno en USA. IMDb la fecha de estreno mundial. Pero sí que coinciden las puntuaciones y la mayoría de películas asignadas a 2019.

In [22]:
movies[['imdb_id', 'title', 'original_title', 'imdb_rating', 'metascore', 'year']][movies.metascore>=82].sort_values('metascore', ascending=False)

,imdb_id,title,original_title,imdb_rating,metascore,year
8861,tt7689960,La lucha pacífica de Martin Luther King,King in the Wilderness,7.9,97.0,2018.0
5297,tt6155172,Roma,NaN,7.7,96.0,2018.0
4005,tt4935462,Amazing Grace,NaN,7.5,94.0,2018.0
10603,tt8075192,Un asunto de familia,Manbiki kazoku,7.9,93.0,2018.0
8210,tt7497366,The Burial of Kojo,NaN,6.5,93.0,2018.0
...,...,...,...,...,...,...
10611,tt8076344,Penguin Highway,Pengin haiwei,7.1,82.0,2018.0
8809,tt7681824,We the Animals,NaN,6.9,82.0,2018.0
10498,tt8055880,Waldheims Walzer,NaN,7.5,82.0,2018.0
10352,tt8026554,Socrates,NaN,7.1,82.0,2018.0


In [23]:
np.sort(movies.metascore.dropna().unique())

array([ 1.,  9., 10., 14., 15., 16., 17., 18., 19., 21., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
       65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77.,
       78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89., 90.,
       91., 92., 93., 94., 96., 97.])

In [24]:
movies.metascore.describe()

count    859.000000
mean      60.257276
std       16.898233
min        1.000000
25%       49.000000
50%       63.000000
75%       73.000000
max       97.000000
Name: metascore, dtype: float64

## `popularity`
Valor que vamos a descartar porque es muy volátil, cambia continuamente. Así, no guarda relación temporal con la taquilla que es el objeto de estudio.

In [25]:
# Convertirlo a número
movies['popularity'] = movies.popularity.str.replace(',', '').astype(float)

In [26]:
movies.popularity.describe()

count     245.000000
mean     2508.604082
std      1433.480824
min        36.000000
25%      1162.000000
50%      2508.000000
75%      3739.000000
max      4981.000000
Name: popularity, dtype: float64

## `directors`, `writers`, `stars`, `countries`, `companies`
Convertir a: comma separated values

In [27]:
movies['directors'] = movies['directors'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

In [28]:
movies['writers'] = movies['writers'].str.replace('[', '', regex=False) \
                                     .str.replace(']', '', regex=False) \
                                     .str.replace("'", "", regex=False) \
                                     .str.replace(", ", ",", regex=False)

In [29]:
movies['stars'] = movies['writers'].str.replace('[', '', regex=False) \
                                   .str.replace(']', '', regex=False) \
                                   .str.replace("'", "", regex=False) \
                                   .str.replace(", ", ",", regex=False)

In [30]:
movies['countries'] = movies['countries'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

In [31]:
movies['companies'] = movies['companies'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

## `opening_us_canada`, `gross_us_canada`, `gross_world`
Convertir estos valores a números, eliminando el signo del dólar

In [32]:
def remove_dolar(amount):
    if str(amount) == 'nan':
        return np.nan
    else:        
        return float(amount.replace(',', '')[1:]) 

In [33]:
movies['gross_us_canada'] = movies['gross_us_canada'].apply(remove_dolar)
movies['gross_us_canada']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
17814   NaN
17815   NaN
17816   NaN
17817   NaN
17818   NaN
Name: gross_us_canada, Length: 17785, dtype: float64

In [34]:
movies['opening_us_canada'] = movies['opening_us_canada'].apply(remove_dolar)
movies['opening_us_canada']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
17814   NaN
17815   NaN
17816   NaN
17817   NaN
17818   NaN
Name: opening_us_canada, Length: 17785, dtype: float64

In [35]:
movies['gross_world'] = movies['gross_world'].apply(remove_dolar)
movies['gross_world']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
17814   NaN
17815   NaN
17816   NaN
17817   NaN
17818   NaN
Name: gross_world, Length: 17785, dtype: float64

## `budget`
### Normalización `budget` a dólares
Son necesarios varios procesos para tener un valor válido de la variable `budget`:
- Separar, con regex, el texto que corresponde a la moneda y el que corresponde a la cantidad (ej. '€195,000'). Estos dos textos serán dos nuevas columnas `budgetCurrency` y `budgetAmount`
- Crear un diccionario con la correspondencia del código usado para las monedas en IMDb y el código ISO
- Crear un archivo externo que contenga las tasas de cambio para cada moneda y año
- Crear una nueva columna `budgetInDolars` con el valor ya normalizado a dólares

In [36]:
import re

def clean_amount_currency(raw_amount):
    
    if str(raw_amount) == 'nan' :
        return (np.nan, np.nan)
    
    # Diccionario con correspondencia del símbolo de moneda de IMDb y el ISO de monedas
    # Monedas de 2019 sin tasas de cambio: 'LVL'
    currency_to_iso = {'$':'USA', 'A$':'AUS', 'ZAR':'ZAF', '₹':'IND', '€':'EU27_2020',
                       'RUR':'RUS', 'R$':'BRA', 'CN¥':'CHN', 'TRL':'TUR', 'CA$':'CAN',
                       'NPR':'NPR', '£':'GBR', '¥':'JPN', 'HUF':'HUN', 'HRK':'HRV',
                       'NOK':'NOR', 'MNT':'MNT', 'CHF':'CHE', 'MX$':'MEX', 'UAH':'UAH',
                       'HK$':'HKG', 'PHP':'PHP', 'PLN':'POL', 'EGP':'EGP' , 'IRR':'IRR',
                       'NZ$':'NZL', 'AED':'AED', 'DKK':'DNK', 'NGN':'NGN', 'SEK':'SWE',
                       'BDT':'BDT', 'MYR':'MYR', 'CZK':'CZE', 'ARS':'ARG', 'PKR':'PKR',
                       'GEL':'GEL', 'NT$':'TWD', '₪':'ISR', '₫':'VND', '₩':'KOR', 'THB':'THB',
                       'RON':'ROU', 'PTE':'PRT', 'TTD':'TTD', 'MVR':'MVR', 'JOD':'JOD',
                       'ISK':'ISL', 'COP':'COL', 'LKR':'LKR', 'BSD':'BSD', 'BGL':'BGR',
                       'KZT':'KZT', 'KES':'KES', 'TZS':'TZS', 'DOP':'DOP', 'TJS':'TJS',
                       'TND':'TND', 'MUR':'MUR', 'SGD':'SGD', 'MOP':'MOP', 'IDR':'IDN',
                       'ETB':'ETB', 'CUP':'CUP', 'BYR':'BYR', 'MMK':'MMK', 'NAD':'NAD',
                       'PEN':'PER', 'MKD':'MKD', 'UGX':'UGX', 'BAM':'BAM', 'ALL':'ALL',
                       'KGS':'KGS', 'GHC':'GHS'
                      }
    
    raw_amount = raw_amount.replace(',', '')
    
    for item in re.finditer(r"^(\D+)(\d+)$", raw_amount):
        try:
            currency = currency_to_iso[item.group(1).strip()]
        except:
            currency = str(np.nan)
        amount = float(item.group(2))

    return(currency, amount)



# Dividir la info de budget en dos nuevas columnas (cantidad y moneda)
movies['budgetCurrency'], movies['budgetAmount'] = zip(*movies['budget'].map(clean_amount_currency))
movies[['title', 'budget', 'budgetAmount', 'budgetCurrency']].dropna()

# importar archivo csv de tasas de cambio
exchange_rates=pd.read_csv('../data/OECD/DP_LIVE_16072021155836489.csv')

# Seleccionar las tasas de cambio para el año del dataset
exchange_rates_anno = exchange_rates[exchange_rates.TIME == anno]

# Merge tasas de cambio con df movies, usando como primary key el código ISO de la moneda
movies = pd.merge(movies, exchange_rates_anno[['LOCATION','Value']], how='left', left_on='budgetCurrency', right_on='LOCATION')
movies['budgetInDollars'] = movies['budgetAmount'] / movies['Value']

movies.head()

,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,...,awards,budget,gross_us_canada,opening_us_canada,gross_world,budgetCurrency,budgetAmount,LOCATION,Value,budgetInDollars
0,tt0069049,Al otro lado del viento,The Other Side of the Wind,2018.0,13,2h 2min,Orson Welles,"Orson Welles,Oja Kodar","Orson Welles,Oja Kodar",['Drama'],...,Awards\n9 wins & 9 nominations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0111414,A Thin Life,NaN,2018.0,TV-PG,1h 15min,Frank Howson,Frank Howson,Frank Howson,['Comedy'],...,NaN,"A$100,000",NaN,NaN,NaN,AUS,100000.0,AUS,1.338412,7.471541e+04
2,tt0137818,Housesitter: The Night They Saved Siegfried's ...,NaN,2018.0,NaN,NaN,Robin Nuyen,"Richard Gasparian,Robin Nuyen","Richard Gasparian,Robin Nuyen",NaN,...,Awards\n1 win & 1 nomination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0170651,T.G.M. - osvoboditel,NaN,2018.0,NaN,NaN,Vera Chytilová,NaN,NaN,['Documentary'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0192528,Reverse Heaven,NaN,2018.0,TV-MA,1h 44min,Stuart Paul,Stuart Paul,Stuart Paul,['Drama'],...,NaN,"$5,000,000",NaN,NaN,NaN,USA,5000000.0,USA,1.000000,5.000000e+06


---
# Merge de tablas


In [37]:
imdb_merged = pd.merge(title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)] , title_ratings, on='tconst', how='left')
imdb_merged

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0.0,2018.0,NaN,122.0,Drama,6.8,6618.0
1,tt0111414,movie,A Thin Life,A Thin Life,0.0,2018.0,NaN,75.0,Comedy,NaN,NaN
2,tt0137818,movie,Housesitter: The Night They Saved Siegfried's ...,Housesitter: The Night They Saved Siegfried's ...,0.0,2018.0,NaN,95.0,NaN,4.4,22.0
3,tt0170651,movie,TGM the Liberator,T.G.M. - osvoboditel,0.0,2018.0,NaN,60.0,Documentary,6.8,14.0
4,tt0192528,movie,Heaven & Hell,Reverse Heaven,0.0,2018.0,NaN,104.0,Drama,4.0,87.0
...,...,...,...,...,...,...,...,...,...,...,...
17806,tt9909940,movie,Hot Scent,Hot Scent,0.0,2018.0,NaN,83.0,"Drama,Family",NaN,NaN
17807,tt9910688,movie,"Luz, Câmera e Barreto","Luz, Câmera e Barreto",0.0,2018.0,NaN,46.0,Documentary,NaN,NaN
17808,tt9914644,movie,9/11: Escape from the Towers,9/11: Escape from the Towers,0.0,2018.0,NaN,120.0,Documentary,8.3,69.0
17809,tt9914662,movie,Wien is 't Hof van Commerce,Wien is 't Hof van Commerce,0.0,2018.0,NaN,NaN,Comedy,NaN,NaN


### Renombrar y seleccionar columnas de ambos datasets

In [38]:
imdb_merged.rename(columns={'primaryTitle': 'englishTitle',
                            'tconst': 'imdbId',
                            'averageRating': 'ratingImdb'
                          }, inplace=True)

In [39]:
imdb_merged.columns

Index(['imdbId', 'titleType', 'englishTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres', 'ratingImdb',
       'numVotes'],
      dtype='object')

In [40]:
movies.rename(columns={'imdb_id': 'imdbId',
                       'title': 'spanishTitle',
                       'budget': 'budgetString',
                       'budgetInDollars': 'budget',
                       'gross_us_canada': 'grossUsCanada',
                       'opening_us_canada': 'openingUsCanada',
                       'gross_world': 'grossWorld',
                      }, inplace=True)

In [41]:
movies.columns

Index(['imdbId', 'spanishTitle', 'original_title', 'year', 'certificate',
       'duration', 'directors', 'writers', 'stars', 'genres', 'countries',
       'companies', 'imdb_rating', 'metascore', 'popularity', 'awards',
       'budgetString', 'grossUsCanada', 'openingUsCanada', 'grossWorld',
       'budgetCurrency', 'budgetAmount', 'LOCATION', 'Value', 'budget'],
      dtype='object')

In [42]:
# Selección de columnas de imdb
imdb_merged = imdb_merged[['imdbId', 'englishTitle', 'originalTitle', 'isAdult', 'runtimeMinutes', 'genres', 'ratingImdb', 'numVotes']]

In [43]:
# Selección de columnas del escrapeado
movies = movies[['imdbId', 'spanishTitle', 'year', 'certificate',
                'directors', 'writers', 'stars', 'countries', 'companies',
                'metascore', 'awards', 'budget', 'grossUsCanada',
                'openingUsCanada', 'grossWorld']]

### Merge final para este año

In [44]:
movies_merged = pd.merge(imdb_merged, movies, on='imdbId', how='inner')
movies_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17777 entries, 0 to 17776
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   imdbId           17777 non-null  object 
 1   englishTitle     17777 non-null  object 
 2   originalTitle    17777 non-null  object 
 3   isAdult          17777 non-null  float64
 4   runtimeMinutes   13883 non-null  float64
 5   genres           17021 non-null  object 
 6   ratingImdb       9653 non-null   float64
 7   numVotes         9653 non-null   float64
 8   spanishTitle     17777 non-null  object 
 9   year             17777 non-null  float64
 10  certificate      2671 non-null   object 
 11  directors        17227 non-null  object 
 12  writers          15696 non-null  object 
 13  stars            15696 non-null  object 
 14  countries        17505 non-null  object 
 15  companies        12850 non-null  object 
 16  metascore        859 non-null    float64
 17  awards      

In [45]:
# Reordenar columnas

movies_merged = movies_merged[['imdbId', 'year', 'spanishTitle', 'originalTitle',
                               'englishTitle', 'ratingImdb', 'numVotes', 'metascore',
                               'isAdult', 'certificate', 'runtimeMinutes', 'genres', 'directors',
                               'writers', 'stars', 'countries', 'companies', 'awards',
                               'budget', 'grossUsCanada', 'openingUsCanada', 'grossWorld']]

In [46]:
movies_merged

,imdbId,year,spanishTitle,originalTitle,englishTitle,ratingImdb,numVotes,metascore,isAdult,certificate,...,directors,writers,stars,countries,companies,awards,budget,grossUsCanada,openingUsCanada,grossWorld
0,tt0069049,2018.0,Al otro lado del viento,The Other Side of the Wind,The Other Side of the Wind,6.8,6618.0,78.0,0.0,13,...,Orson Welles,"Orson Welles,Oja Kodar","Orson Welles,Oja Kodar","France,Iran,United States","Royal Road Entertainment,""Les Films de lAstrop...",Awards\n9 wins & 9 nominations,NaN,NaN,NaN,NaN
1,tt0111414,2018.0,A Thin Life,A Thin Life,A Thin Life,NaN,NaN,NaN,0.0,TV-PG,...,Frank Howson,Frank Howson,Frank Howson,Australia,Boulevard Films,NaN,7.471541e+04,NaN,NaN,NaN
2,tt0137818,2018.0,Housesitter: The Night They Saved Siegfried's ...,Housesitter: The Night They Saved Siegfried's ...,Housesitter: The Night They Saved Siegfried's ...,4.4,22.0,NaN,0.0,NaN,...,Robin Nuyen,"Richard Gasparian,Robin Nuyen","Richard Gasparian,Robin Nuyen",United States,Basement Productions,Awards\n1 win & 1 nomination,NaN,NaN,NaN,NaN
3,tt0170651,2018.0,T.G.M. - osvoboditel,T.G.M. - osvoboditel,TGM the Liberator,6.8,14.0,NaN,0.0,NaN,...,Vera Chytilová,NaN,NaN,Czech Republic,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0192528,2018.0,Reverse Heaven,Reverse Heaven,Heaven & Hell,4.0,87.0,NaN,0.0,TV-MA,...,Stuart Paul,Stuart Paul,Stuart Paul,United States,SP Sales Worldwide,NaN,5.000000e+06,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17772,tt9909940,2018.0,Hot Scent,Hot Scent,Hot Scent,NaN,NaN,NaN,0.0,NaN,...,Ali Ebrahimi,"Ali Hamidnejad,Ali Shoorvarzi","Ali Hamidnejad,Ali Shoorvarzi",Iran,Art House,NaN,NaN,NaN,NaN,NaN
17773,tt9910688,2018.0,"Luz, Câmera e Barreto","Luz, Câmera e Barreto","Luz, Câmera e Barreto",NaN,NaN,NaN,0.0,NaN,...,"Vinicius Augusto Bozzo,Angela Gurgel","Vinicius Augusto Bozzo,Angela Gurgel","Vinicius Augusto Bozzo,Angela Gurgel",Brazil,TV Assembleia do Ceará,NaN,5.473716e+03,NaN,NaN,NaN
17774,tt9914644,2018.0,9/11: Escape from the Towers,9/11: Escape from the Towers,9/11: Escape from the Towers,8.3,69.0,NaN,0.0,NaN,...,Grace Chapman,NaN,NaN,United Kingdom,Arrow Media,NaN,NaN,NaN,NaN,NaN
17775,tt9914662,2018.0,Wien is 't Hof van Commerce,Wien is 't Hof van Commerce,Wien is 't Hof van Commerce,NaN,NaN,NaN,0.0,NaN,...,Kristof Michiels,Kristof Michiels,Kristof Michiels,Belgium,NaN,NaN,NaN,NaN,NaN,NaN


## Feature engineering. Columnas `profit` y `roi`
Los estadísticos están distorsionados por los valores nulos que se eliminarán después

In [47]:
movies_merged['profit'] = movies_merged.grossWorld - movies_merged.budget
movies_merged['profit'].describe()

count    6.690000e+02
mean     3.214455e+07
std      1.382886e+08
min     -1.060089e+08
25%     -2.334377e+06
50%     -4.292920e+05
75%      2.442161e+06
max      1.727360e+09
Name: profit, dtype: float64

In [48]:
movies_merged['roi'] = (movies_merged.grossWorld - movies_merged.budget) / movies_merged.budget
movies_merged['roi'].describe()

count    6.690000e+02
mean     1.876200e+04
std      4.133976e+05
min     -9.999538e-01
25%     -9.613957e-01
50%     -6.690052e-01
75%      9.025429e-01
max      1.060861e+07
Name: roi, dtype: float64

---
# Inspección inicial de correlaciones
## Registros no nulos con los que hacer las correlaciones de valoraciones y recaudación
Valoraciones: imdb_rating, metascore

Recaudación: budget, gross_world


In [49]:
print('Registros con ratingImdb:', len(movies_merged[movies_merged.ratingImdb.notnull()]))
print('Registros con metascore:', len(movies_merged[movies_merged.metascore.notnull()]))

print('Registros con budget:', len(movies_merged[movies_merged.budget.notnull()]))
print('Registros con grossUsCanada:', len(movies_merged[movies_merged.grossUsCanada.notnull()]))
print('Registros con openingUsCanada:', len(movies_merged[movies_merged.openingUsCanada.notnull()]))
print('Registros con grossWorld:', len(movies_merged[movies_merged.grossWorld.notnull()]))

print('\nRegistros con las rating, metascore, budget, grossWorld:',len(movies_merged[movies_merged.budget.notnull() & movies_merged.grossWorld.notnull() & movies_merged.ratingImdb.notnull() & movies_merged.metascore.notnull()]))

Registros con ratingImdb: 9653
Registros con metascore: 859
Registros con budget: 4876
Registros con grossUsCanada: 731
Registros con openingUsCanada: 650
Registros con grossWorld: 3033

Registros con las rating, metascore, budget, grossWorld: 241


In [50]:
movies_merged = movies_merged[movies_merged.budget.notnull() & movies_merged.grossWorld.notnull() & movies_merged.ratingImdb.notnull() & movies_merged.metascore.notnull()]
movies_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 13 to 17306
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   imdbId           241 non-null    object 
 1   year             241 non-null    float64
 2   spanishTitle     241 non-null    object 
 3   originalTitle    241 non-null    object 
 4   englishTitle     241 non-null    object 
 5   ratingImdb       241 non-null    float64
 6   numVotes         241 non-null    float64
 7   metascore        241 non-null    float64
 8   isAdult          241 non-null    float64
 9   certificate      228 non-null    object 
 10  runtimeMinutes   241 non-null    float64
 11  genres           241 non-null    object 
 12  directors        241 non-null    object 
 13  writers          241 non-null    object 
 14  stars            241 non-null    object 
 15  countries        241 non-null    object 
 16  companies        240 non-null    object 
 17  awards       

## Correlaciones de valoraciones y recaudación

In [51]:
movies_merged.describe().round(2)

,year,ratingImdb,numVotes,metascore,isAdult,runtimeMinutes,budget,grossUsCanada,openingUsCanada,grossWorld,profit,roi
count,241.0,241.00,241.00,241.00,241.0,241.00,2.410000e+02,2.020000e+02,1.980000e+02,2.410000e+02,2.410000e+02,241.00
mean,2018.0,6.39,70007.95,56.63,0.0,109.07,3.253096e+07,5.507352e+07,1.707979e+07,1.195616e+08,8.703063e+07,618.64
std,0.0,0.91,115818.99,17.99,0.0,17.10,4.836808e+07,1.011128e+08,3.277769e+07,2.541895e+08,2.160229e+08,9502.98
min,2018.0,3.20,93.00,1.00,0.0,60.00,1.299000e+01,9.740000e+02,6.550000e+02,8.530000e+02,-3.167364e+07,-1.00
25%,2018.0,5.80,7009.00,44.00,0.0,97.00,5.000000e+06,2.527645e+06,2.100148e+05,2.053469e+06,-3.001596e+06,-0.66
50%,2018.0,6.50,29134.00,58.00,0.0,107.00,1.300000e+07,1.920107e+07,6.571880e+06,2.078068e+07,5.515914e+06,0.58
75%,2018.0,7.00,71328.00,69.00,0.0,119.00,3.700000e+07,5.482679e+07,1.747564e+07,9.533071e+07,7.254213e+07,3.01
max,2018.0,8.40,893456.00,92.00,0.0,158.00,3.210000e+08,7.004266e+08,2.576982e+08,2.048360e+09,1.727360e+09,147531.03


In [52]:
method = 'pearson'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2018 Corr Método: pearson


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.705652,0.131369,0.240412,0.253473,0.064908
metascore,0.705652,1.000000,0.041181,0.105312,0.114698,-0.016278
budget,0.131369,0.041181,1.000000,0.824988,0.746843,-0.043777
grossWorld,0.240412,0.105312,0.824988,1.000000,0.991962,-0.029967
profit,0.253473,0.114698,0.746843,0.991962,1.000000,-0.025460
roi,0.064908,-0.016278,-0.043777,-0.029967,-0.025460,1.000000


In [53]:
method = 'kendall'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2018 Corr Método: kendall


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.538208,-0.000956,0.085559,0.147266,0.133546
metascore,0.538208,1.000000,-0.084135,-0.025857,0.080150,0.020700
budget,-0.000956,-0.084135,1.000000,0.607312,0.288472,0.220294
grossWorld,0.085559,-0.025857,0.607312,1.000000,0.683057,0.615284
profit,0.147266,0.080150,0.288472,0.683057,1.000000,0.626279
roi,0.133546,0.020700,0.220294,0.615284,0.626279,1.000000


In [54]:
method = 'spearman'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2018 Corr Método: spearman


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.713776,-0.003884,0.123142,0.212732,0.191023
metascore,0.713776,1.000000,-0.129988,-0.046680,0.112186,0.029290
budget,-0.003884,-0.129988,1.000000,0.792413,0.453621,0.298137
grossWorld,0.123142,-0.046680,0.792413,1.000000,0.842762,0.775197
profit,0.212732,0.112186,0.453621,0.842762,1.000000,0.835200
roi,0.191023,0.029290,0.298137,0.775197,0.835200,1.000000


In [55]:
movies_merged[['ratingImdb', 'metascore', 'budget', 'grossUsCanada', 'openingUsCanada', 'grossWorld', 'profit', 'roi']].corr()

,ratingImdb,metascore,budget,grossUsCanada,openingUsCanada,grossWorld,profit,roi
ratingImdb,1.000000,0.705652,0.131369,0.256055,0.196398,0.240412,0.253473,0.064908
metascore,0.705652,1.000000,0.041181,0.164138,0.105374,0.105312,0.114698,-0.016278
budget,0.131369,0.041181,1.000000,0.776146,0.783215,0.824988,0.746843,-0.043777
grossUsCanada,0.256055,0.164138,0.776146,1.000000,0.969052,0.940519,0.931294,-0.028704
openingUsCanada,0.196398,0.105374,0.783215,0.969052,1.000000,0.927954,0.914799,-0.030812
grossWorld,0.240412,0.105312,0.824988,0.940519,0.927954,1.000000,0.991962,-0.029967
profit,0.253473,0.114698,0.746843,0.931294,0.914799,0.991962,1.000000,-0.025460
roi,0.064908,-0.016278,-0.043777,-0.028704,-0.030812,-0.029967,-0.025460,1.000000


---
# Guardado dataset a pickle

In [56]:
import pickle

with open('../data/web_imdb_clean/movies_2018.pickle', 'wb') as f:
    pickle.dump(movies_merged, f)